# Hands-on: Question Answering by Querying PA Data

## Overview

This Jupyter Notebook provides an example of how to:

1. Develop a Question Answering application that can answer user's question based on Planning Analytics data.

2. Construct a One-Shot Prompt and pass the prompt to Large Language Model (LLM) to generate MDX statement based on user's question.

3. Construct another Prompt and pass the retrieved tabular dataset to Large Language Model (LLM) to generate a summary of the data.

In [1]:
# Install library
%pip install TM1py

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Import libraries
import json
import os
import random
import requests
from TM1py.Services import TM1Service
from TM1py.Exceptions import TM1pyException
from TM1py.Utils.Utils import build_pandas_dataframe_from_cellset
from TM1py.Utils.Utils import build_cellset_from_pandas_dataframe

from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

# WML python SDK
from ibm_watson_machine_learning.foundation_models import Model
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes, DecodingMethods

from TM1py.Services import TM1Service
from TM1py.Exceptions import TM1pyException

## 1. Setup Connection to Planning Analytics

To get connected to Planning Analytics server, you need these informations:
- address
- port
- user
- password
- namespace

Your connection is successful when you see the *Server Name* and *Product Version* information.

In [3]:
#Set up connection to Planning Analytics server
try:
    with TM1Service(
        address='na4.services.cloud.techzone.ibm.com', #'<YOUR PA SERVER ADDRESS HERE>',
        port= 31758, #<YOUR PA PORT NUMBER HERE>,
        ssl=False,
        user="pm",
        password="IBMDem0s",
        namespace='Harmony LDAP'
    ) as tm1:
        print("Server Name:", tm1.server.get_server_name())
        print("Product Version:",tm1.server.get_product_version())

# Error Handling        
except TM1pyException as e:
    if e.status_code == 401:
        print('Wrong credentials')
    elif e.status_code == 404:
        print('Wrong connection')
    else:
        print('Something else went wrong. Check error code:', str(e))

Server Name: 24Retail
Product Version: 11.8.02300.10


## 2. User's Question

User's question in natural language.

In [4]:
question = "How many units of product 21002 did we sell in year 1 in organization 102 through channel 10?"

## 3. Configuring watsonx.ai

The following section defines the input to the Large Language Model (LLM).
Provides the credential for watsonx.ai as indicated below

1. `watsonx_project_id`: The watsonx.ai **Project ID** provided in watsonx.ai project -> Manage -> Project id
2. `api_key`: The **API Key** provided in IBM Cloud -> Manage -> API Key

In [5]:
project_id = os.environ["PROJECT_ID"]

In [6]:
# URL of the hosted LLMs is hardcoded because at this time all LLMs share the same endpoint
url = "https://us-south.ml.cloud.ibm.com"

# Replace with your watsonx project id (look up in the project Manage tab)
watsonx_project_id = "d7661c12-f79b-4bd6-96f7-418e9e2d561c"  #"<YOUR WATSONX.AI PROJECT ID HERE>"

# Replace with your IBM Cloud key
api_key = "4k3SWfqY13YcE1megHvu5q5HeY1tyjY9tqk8vs7C1HHq" #"<YOUR IBM CLOUD API KEY HERE>"

In [11]:
model_init = None

def get_model_mdx(model_type, max_tokens, min_tokens, decoding, stop_sequences):
    generate_params = {
        GenParams.MAX_NEW_TOKENS: max_tokens,
        GenParams.MIN_NEW_TOKENS: min_tokens,
        GenParams.DECODING_METHOD: decoding,
        GenParams.STOP_SEQUENCES: stop_sequences
    }
    global model_init
    if model_init is None:
        model_init = Model(
            model_id=model_type,
            params=generate_params,
            credentials={
                "apikey": api_key,
                "url": url
            },
            project_id=watsonx_project_id
        )

    return model_init

In [12]:
model_init = None

def get_model (model_type, max_tokens, min_tokens, decoding, repetition_penalty):
    generate_params = {
        GenParams.MAX_NEW_TOKENS: max_tokens,
        GenParams.MIN_NEW_TOKENS: min_tokens,
        GenParams.DECODING_METHOD: decoding,
        GenParams.REPETITION_PENALTY: repetition_penalty
    }
    global model_init
    if model_init is None:
        model_init = Model(
            model_id=model_type,
            params=generate_params,
            credentials={
                "apikey": api_key,
                "url": url
            },
            project_id=watsonx_project_id
        )

    return model_init

## 4. Creating Prompt to generate MDX statement to query data in PA

- Construct a One-Shot Prompt and pass the prompt to Large Language Model (LLM) to generate MDX statement.
- Based on user's question in natural language, LLM will generate the MDX statement.
- The MDX statement will be used to query data in Planning Analytics.

In [13]:
prompt_mdx = """Create an MDX Statement for a Planning Analytics View to display the desired data

Input:
How many units of product 21001 did we sell in year 2 in organization 101 through channel 10?

Output:
SELECT {[Revenue].[Revenue].[Units Sold]} ON 0, {[product].[product].[21001]}*{DRILLDOWNMEMBER({[Year].[Year].[Y2]} , {[Year].[Year].[Y2]})}*{TM1SubsetToSet([Month].[Month],"MY","public")} ON 1 FROM [Revenue] WHERE ([organization].[organization].[101], [Channel].[Channel].[10], [Version].[Version].[Actual])

Input:
""" + str(question) + """

Output:
"""
print(prompt_mdx)

Create an MDX Statement for a Planning Analytics View to display the desired data

Input:
How many units of product 21001 did we sell in year 2 in organization 101 through channel 10?

Output:
SELECT {[Revenue].[Revenue].[Units Sold]} ON 0, {[product].[product].[21001]}*{DRILLDOWNMEMBER({[Year].[Year].[Y2]} , {[Year].[Year].[Y2]})}*{TM1SubsetToSet([Month].[Month],"MY","public")} ON 1 FROM [Revenue] WHERE ([organization].[organization].[101], [Channel].[Channel].[10], [Version].[Version].[Actual])

Input:
How many units of product 21002 did we sell in year 1 in organization 102 through channel 10?

Output:



## 4. Model Parameter for LLM to generate MDX statement to query data in PA

In [14]:
# Set up watsonx model and parameters
model_type = "codellama/codellama-34b-instruct-hf" # Coding model
max_tokens = 200
min_tokens = 10
decoding = DecodingMethods.GREEDY
stop_sequences = ["\n\n"]

# Get the watsonx model
model_mdx = get_model_mdx(model_type, max_tokens, min_tokens, decoding, stop_sequences)

## 4. LLM Inferencing to generate MDX statement to query data in PA

In [15]:
# Send a prompt to model
generated_response = model_mdx.generate(prompt_mdx)
response_mdx = generated_response['results'][0]['generated_text']

# Print model response
print("--------------------------------- Generated response -----------------------------------")
print(response_mdx)

--------------------------------- Generated response -----------------------------------
SELECT {[Revenue].[Revenue].[Units Sold]} ON 0, {[product].[product].[21002]}*{DRILLDOWNMEMBER({[Year].[Year].[Y1]} , {[Year].[Year].[Y1]})}*{TM1SubsetToSet([Month].[Month],"MY","public")} ON 1 FROM [Revenue] WHERE ([organization].[organization].[102], [Channel].[Channel].[10], [Version].[Version].[Actual])




## 4. Query data in PA based on the AI-generated MDX statement

In [16]:
data = tm1.cubes.cells.execute_mdx(mdx=response_mdx, private=False, use_compact_json=True)
df = build_pandas_dataframe_from_cellset(data, multiindex=False, sort_values=False)
df

,organization,Channel,product,Month,Year,Version,Revenue,Values
0,102,10,21002,Jan,Y1,Actual,Units Sold,246.161436
1,102,10,21002,Feb,Y1,Actual,Units Sold,239.335100
2,102,10,21002,Mar,Y1,Actual,Units Sold,233.672426
3,102,10,21002,Apr,Y1,Actual,Units Sold,222.258007
4,102,10,21002,May,Y1,Actual,Units Sold,204.470813
5,102,10,21002,Jun,Y1,Actual,Units Sold,178.039069
6,102,10,21002,Jul,Y1,Actual,Units Sold,93.295976
7,102,10,21002,Aug,Y1,Actual,Units Sold,60.882947
8,102,10,21002,Sep,Y1,Actual,Units Sold,59.570892
9,102,10,21002,Oct,Y1,Actual,Units Sold,38.951568


In [17]:
# Convert the cellset into a list of dictionaries
data_list = [{item: value} for item, value in data.items()]
df = df.rename({'Revenue': 'KPI'}, axis=1)
df_md = df.to_markdown(index=False)
print(df_md)

|   organization |   Channel |   product | Month   | Year   | Version   | KPI        |    Values |
|---------------:|----------:|----------:|:--------|:-------|:----------|:-----------|----------:|
|            102 |        10 |     21002 | Jan     | Y1     | Actual    | Units Sold |  246.161  |
|            102 |        10 |     21002 | Feb     | Y1     | Actual    | Units Sold |  239.335  |
|            102 |        10 |     21002 | Mar     | Y1     | Actual    | Units Sold |  233.672  |
|            102 |        10 |     21002 | Apr     | Y1     | Actual    | Units Sold |  222.258  |
|            102 |        10 |     21002 | May     | Y1     | Actual    | Units Sold |  204.471  |
|            102 |        10 |     21002 | Jun     | Y1     | Actual    | Units Sold |  178.039  |
|            102 |        10 |     21002 | Jul     | Y1     | Actual    | Units Sold |   93.296  |
|            102 |        10 |     21002 | Aug     | Y1     | Actual    | Units Sold |   60.8829 |
|         

## 4. Model Parameter for LLM to generate MDX statement to query data in PA

In [36]:
# Set up watsonx model and parameters
model_type = 'ibm/granite-13b-instruct-v2'
#model_type = "meta-llama/llama-2-70b-chat"
max_tokens = 1000
#min_tokens = 10
decoding = DecodingMethods.GREEDY
#temperature = 0.7
repetition_penalty = 1

# Get the watsonx model
model_anwser = get_model(model_type, max_tokens, min_tokens, decoding, repetition_penalty)

## 4. Creating Prompt to generate MDX statement to query data in PA

- Construct a One-Shot Prompt and pass the prompt to Large Language Model (LLM) to generate MDX statement.
- Based on user's question in natural language, LLM will generate the MDX statement.
- The MDX statement will be used to query data in Planning Analytics.

In [72]:
prompt = """<s>[INST] <<SYS>> Answer the question with the information contained in the following table. If the question is unanswerable, say 'unanswerable'. <</SYS>> Question: """ + question + df_md + '[/INST]'

print(prompt)

<s>[INST] <<SYS>> Answer the question with the information contained in the following table. If the question is unanswerable, say 'unanswerable'. <</SYS>> Question: How many units of product 21002 did we sell in year 1 in organization 102 through channel 10?|   organization |   Channel |   product | Month   | Year   | Version   | KPI        |    Values |
|---------------:|----------:|----------:|:--------|:-------|:----------|:-----------|----------:|
|            102 |        10 |     21002 | Jan     | Y1     | Actual    | Units Sold |  246.161  |
|            102 |        10 |     21002 | Feb     | Y1     | Actual    | Units Sold |  239.335  |
|            102 |        10 |     21002 | Mar     | Y1     | Actual    | Units Sold |  233.672  |
|            102 |        10 |     21002 | Apr     | Y1     | Actual    | Units Sold |  222.258  |
|            102 |        10 |     21002 | May     | Y1     | Actual    | Units Sold |  204.471  |
|            102 |        10 |     21002 | Jun   

## 4. LLM Inferencing to generate answer based on the data queried from PA

In [73]:
# Send a prompt to model
generated_response = model_anwser.generate(prompt)
response_text = generated_response['results'][0]['generated_text']

# Print model response
print("--------------------------------- Generated response -----------------------------------")
print(response_text)

--------------------------------- Generated response -----------------------------------
  The answer to the question is 1645.85. This is the total number of units of product 21002 sold by organization 102 through channel 10 in year 1, as shown in the table.
